# NeRF_PL

## 1. Run COLMAP
Input data in `$DATASET_PATH/images`

In [6]:
import os

os.environ['DATASET_PATH'] = "/workspace/colmap"

!colmap automatic_reconstructor \
    --workspace_path $DATASET_PATH \
    --image_path $DATASET_PATH/images


Feature extraction

Processed file [1/16]
  Name:            10.png
  Dimensions:      6192 x 8256
  Camera:          #2 - SIMPLE_RADIAL
  Focal Length:    9907.20px
  Features:        5434
Processed file [2/16]
  Name:            1.png
  Dimensions:      6192 x 8256
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    9907.20px
  Features:        10598
Processed file [3/16]
  Name:            11.png
  Dimensions:      6192 x 8256
  Camera:          #3 - SIMPLE_RADIAL
  Focal Length:    9907.20px
  Features:        4864
Processed file [4/16]
  Name:            12.png
  Dimensions:      6192 x 8256
  Camera:          #4 - SIMPLE_RADIAL
  Focal Length:    9907.20px
  Features:        8218
Processed file [5/16]
  Name:            13.png
  Dimensions:      6192 x 8256
  Camera:          #5 - SIMPLE_RADIAL
  Focal Length:    9907.20px
  Features:        12418
Processed file [6/16]
  Name:            14.png
  Dimensions:      6192 x 8256
  Camera:          #6 - SIMPLE_RADIAL
  Focal Len

## 2. Creating a dataset in LLFF format

In [8]:
%cd /workspace/LLFF

!python imgs2poses.py $DATASET_PATH

/workspace/LLFF
Don't need to run COLMAP
Post-colmap
Cameras 5
Images # 16
Points (5043, 3) Visibility (5043, 16)
Depth stats 2.1826621059477502 4483.712710067064 18.256303716610194
Done with imgs2poses


## 3. Train NeRF_PL 
360 inward-facing scene

In [9]:
%cd /workspace/nerf_pl

# set training configurations here
os.environ['IMG_W'] = "1080" # image width (do not set too large)
os.environ['IMG_H'] = "1440" # image height (do not set too large)
os.environ['NUM_EPOCHS'] = "1" # number of epochs to train (depending on how many images there are,
                                # 20~30 might be enough)
os.environ['EXP'] = "exp" # name of the experience (arbitrary)

!python train.py \
   --dataset_name llff \
   --root_dir $DATASET_PATH \
   --N_importance 64 --img_wh $IMG_W $IMG_H \
   --spheric --use_disp \
   --num_epochs $NUM_EPOCHS --batch_size 1024 \
   --optimizer adam --lr 5e-4 \
   --lr_scheduler cosine \
   --exp_name $EXP

/workspace/nerf_pl
GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
val image is /workspace/colmap/images/11.png
/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_war

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch 1: 100%|▉| 22782/22783 [1:36:54<00:00,  3.92it/s, loss=0.035, train_psnr=1
Validating: 0it [00:00, ?it/s]/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

Epoch 1: 100%|█| 22783/22783 [1:39:40<00:00,  3.81it/s, loss=0.035, train_psnr=1
Epoch 1: 100%|█| 22783/22783 [1:39:40<00:00,  3.81it/s, loss=0.035, train_psnr=1


Profiler Report

Action              	|  Mean duration (s)	|  Total time (s) 
-----------------------------------------------------------------
on_train_start      	|  0.007086       	|  0.007086       
on_epoch_start      	|  0.010683       	|  0.01

## 4. Test NeRF_PL

In [ ]:
os.environ['SCENE'] = 'test'
os.environ['CKPT_PATH'] = '/workspace/nerf_pl/ckpts/exp/epoch=0_v0.ckpt'

!python eval.py \
   --root_dir $DATASET_PATH \
   --dataset_name llff --scene_name $SCENE \
   --img_wh $IMG_W $IMG_H --N_importance 64 --ckpt_path $CKPT_PATH

/opt/conda/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
  8%|███                                     | 9/120 [22:33<4:43:30, 153.24s/it]